In [1]:
# 참고 사이트
# KoELECTRA
# https://github.com/monologg/KoELECTRA

# Chatbot_data
# https://github.com/songys/Chatbot_data

In [2]:
# 필요 library
!pip install transformers

# Chatbot data
!git clone https://github.com/songys/Chatbot_data.git

     |████████████████████████████████| 2.1MB 5.6MB/s 
     |████████████████████████████████| 3.3MB 28.8MB/s 
     |████████████████████████████████| 870kB 38.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=50a8908313d9264cbf64d8c33e6ba25c602e9df28d2c54c764d339a040a3247c
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
Cloning into 'Chatbot_data'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 42 (delta 14), reused 0 (delta 0), pack-reused 18
Unpacking objects: 100% (42/42), done.


In [27]:
import torch
import pandas as pd
import numpy as np
from collections import defaultdict
from transformers import ElectraModel, ElectraTokenizer

In [4]:
MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
model = ElectraModel.from_pretrained(MODEL_NAME)
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)

In [5]:
chatbot_data = pd.read_csv("/content/Chatbot_data/ChatbotData .csv")
print("chatbot_data shape :", chatbot_data.shape)

# label: 0 일상다반사
print("label이 0인 data shape :", chatbot_data[chatbot_data["label"] == 0].shape)
chatbot_data[chatbot_data["label"] == 0].head(2)

chatbot_data shape : (11823, 3)
label이 0인 data shape : (5290, 3)


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0


In [6]:
# label: 1 이별(부정)
print("label이 1인 data shape :", chatbot_data[chatbot_data["label"] == 1].shape)
chatbot_data[chatbot_data["label"] == 1].head(2)

label이 1인 data shape : (3570, 3)


,Q,A,label
5290,1000일 만난 여자친구와 이별,더 오래 만날 사람 만날 거예요.,1
5291,10년 연애. 헤어졌습니다.,더 공허함이 크시겠네요.,1


In [7]:
# label: 2 사랑(긍정)
print("label이 2인 data shape :", chatbot_data[chatbot_data["label"] == 2].shape)
chatbot_data[chatbot_data["label"] == 2].head(2)

label이 2인 data shape : (2963, 3)


,Q,A,label
8860,짝사랑만큼 고통스러운 건 없겠지.,짝사랑 만큼 감정소모가 큰 건 없을 거예요.,2
8861,1년 넘게 만났는데 지금도 불타올라,정열적인 사랑을 하고 있나봐요.,2


### 일상다반사만을 이용한 Chatbot

In [8]:
data = chatbot_data[chatbot_data["label"] == 0]
data.head(2)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0


In [9]:
length_count_dict = defaultdict(lambda: 0)

def length_count(text):
    length_count_dict[len(text["Q"])] += 1
    length_count_dict[len(text["A"])] += 1

data.apply(length_count, axis=1)

# max_length: 50
length_count_dict

defaultdict(<function __main__.<lambda>>,
            {1: 8,
             2: 21,
             3: 61,
             4: 145,
             5: 243,
             6: 547,
             7: 643,
             8: 861,
             9: 1029,
             10: 1060,
             11: 895,
             12: 879,
             13: 822,
             14: 690,
             15: 543,
             16: 461,
             17: 332,
             18: 297,
             19: 222,
             20: 171,
             21: 138,
             22: 90,
             23: 81,
             24: 58,
             25: 55,
             26: 36,
             27: 32,
             28: 40,
             29: 14,
             30: 27,
             31: 18,
             32: 11,
             33: 4,
             34: 8,
             35: 4,
             36: 4,
             38: 4,
             39: 4,
             40: 2,
             41: 4,
             44: 5,
             45: 3,
             48: 4,
             50: 4})

In [10]:
def test(text):
    return tokenizer.decode(tokenizer.encode(text))

data["Q"].apply(test).head()

0             [CLS] 12시 땡! [SEP]
1        [CLS] 1지망 학교 떨어졌어 [SEP]
2       [CLS] 3박4일 놀러가고 싶다 [SEP]
3    [CLS] 3박4일 정도 놀러가고 싶다 [SEP]
4            [CLS] PPL 심하네 [SEP]
Name: Q, dtype: object

In [11]:
def get_cls_token(sent):
    model.eval()
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        add_special_tokens=True
    )

    with torch.no_grad():
        outputs = model(**tokenized_sent) # keyword argument

    # pooler_output / last_hidden_state[:,0,:] 동일한지?
    logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
    return logits

In [15]:
data["Q_cls_token"] = data["Q"].apply(get_cls_token)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
def cos_sim(A, B):
    return np.dot(A, B)/(np.linalg.norm(A)*np.linalg.norm(B))

while True:
    text = input()
    if text == "그만":
        break
    text_token = get_cls_token(text)
    print(data[data.index == data["Q_cls_token"].apply(lambda x: cos_sim(x[0], text_token[0])).argmax()]["A"].values[0])

안녕
안녕하세요.
반가워
저도 반가워요.
이제 할 일 끝났어
안전 운전 하세요.
나 집인데?
지극히 평범하면서 지극히 특별하죠.
시간이 늦어 자야될 거 같아
부모님께 도움을 청해보세요.
나 이제 자도 돼?
저는 배터리가 밥이예요.
잠 잘 오겠지?
야근이 만든 야경이죠.
난 잘게
안녕히 주무세요.
그만
